In [ ]:
#!/usr/bin/env python
from numcosmo_py import Ncm, Nc
from numcosmo_py.external.pyssc import pyssc as PySSC
from  numcosmo_py.ccl import nc_ccl

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())
import numpy as np
import math
from astropy.io import fits
from astropy.table import Table
import sys
sys.path.insert(0,'/home/henrique/cosmolibraries/CLCosmo_Sim/cluster_abundance/')
sys.path.insert(0,'../../scripts')

import pyccl as ccl
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
#import CL_COUNT_class_cluster_abundance as cl_count
import time
#import CL_COUNT_modeling_richness as richness_model

initialize the `Cosmological models`

In [ ]:
# CCL cosmology
cosmo_ccl = ccl.Cosmology(
    Omega_c=0.265,
    Omega_b=0.0448,
    h=0.71,
    sigma8=0.8,
    n_s=0.96,
    Omega_k=0,
    transfer_function="eisenstein_hu",
)
#nc_ccl.set_high_prec_params (cosmo_ccl)

cosmo, dist, ps_lin, ps_nln, psf = nc_ccl.create_nc_obj(cosmo_ccl)

initialize the `ClusterAbundance` object

In [ ]:
# Numcosmo Cluster Abundance

# First we need to define the multiplicity function here we will use the tinker
mulf = Nc.MultiplicityFuncTinker.new()
mulf.set_linear_interp(True)
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
# Second we need to construct a filtered power spectrum

hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area(5264.05 * (np.pi / 180) ** 2)

# Here we define the mass proxy in the first analysis is used the true mass and redshift of the clusters
cluster_m = Nc.ClusterMassNodist(lnM_min=np.log(10) * 14.0, lnM_max=np.log(10) * 16.0)
cluster_z = Nc.ClusterRedshiftNodist(z_min=0.0, z_max=2.0)

ca = Nc.ClusterAbundance.new(hmf, None)
mset = Ncm.MSet.new_array([cosmo, cluster_m, cluster_z])
mset.pretty_log()
print("sigma8 = " + str(cosmo.sigma8(psf)))

In [ ]:
# CCL Cluster Abundance
clc = cl_count.ClusterAbundance()
# choose the halo mass function and mass definition
massdef = ccl.halos.massdef.MassDef(200, "critical", c_m_relation=None)
hmd = ccl.halos.hmfunc.MassFuncTinker08(cosmo_ccl, mass_def=massdef)
clc.set_cosmology(cosmo=cosmo_ccl, hmd=hmd, massdef=massdef)
clc.sky_area = 5264.05 * (np.pi / 180) ** 2
# 0.4 segundos
# 23.3 segundos high precision

# Binned approach
Compute the 2D integral

$$
N_{\alpha\beta}^{\rm predicted} = \Omega_s\int_{z_\alpha}^{z_{\alpha + 1}} dz\int_{\log_{10}M_\beta}^{\log_{10}M_{\beta + 1}}\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}d\log_{10}M
$$

In [ ]:
t0 = time.time()
# Bins definition
z_nodes = np.linspace(0.25, 2, 8)
log10M_nodes = np.linspace(14, 14.8, 11)
lnM_nodes = log10M_nodes * math.log(10)

# Numcosmo cluster counts in the bins

Numcosmo_Abundance = []
ca.prepare(cosmo, cluster_z, cluster_m)
for zl, zu in zip(z_nodes, z_nodes[1:]):
    nc_bin_mass = []
    for lnMl, lnMu in zip(lnM_nodes, lnM_nodes[1:]):
        Pbin = ca.intp_bin_d2n(
            cosmo, cluster_z, cluster_m, [lnMl], [lnMu], None, [zl], [zu], None
        )
        nc_bin_mass.append(Pbin)
    Numcosmo_Abundance.append(nc_bin_mass)

# Numcosmo_Abundance[i][j] is the number of clusters in i-th z_bin and j-th mass_bin


t1 = time.time()
print("time (seconds) = " + str(t1 - t0))

In [ ]:
# CCL cluster count in the bins
t0 = time.time()
CCL_Abundance_exact = clc.Cluster_Abundance_MZ(
    zbin_edges=z_nodes, proxybin_edges=log10M_nodes, method="exact"
)
t1 = time.time()
print("time (seconds) = " + str(t1 - t0))
# CCL_Abundance_exact[i][j] is the number of clusters in i-th z_bin and j-th mass_bin
# 29.50 segundos
# 21.21 segundos high precision

In [ ]:
for zl, zu, ccl_a_z, nc_a_z in zip(
    z_nodes, z_nodes[1:], CCL_Abundance_exact, Numcosmo_Abundance
):
    for lnMl, lnMu, ccl_a, nc_a in zip(lnM_nodes, lnM_nodes[1:], ccl_a_z, nc_a_z):
        print(
            "(% 16.9g % 16.9g) (% 16.9g % 16.9g) % 16.9g cmp %.2e"
            % (
                lnMl / math.log(10),
                lnMu / math.log(10),
                zl,
                zu,
                ccl_a,
                math.fabs(nc_a / ccl_a - 1.0),
            )
        )

In [ ]:
diff = []
for i in range(len(CCL_Abundance_exact)):
    diff.append(
        100
        * abs((Numcosmo_Abundance[i] - CCL_Abundance_exact[i]) / Numcosmo_Abundance[i])
    )

plt.figure(figsize=(14, 9))
plt.yscale("log")
z_ = [np.mean([z_nodes[i], z_nodes[i + 1]]) for i in np.arange(len(z_nodes) - 1)]
mass = [
    np.mean([10 ** log10M_nodes[i], 10 ** log10M_nodes[i + 1]])
    for i in np.arange(len(log10M_nodes) - 1)
]
plt.tick_params(axis="both", which="both", labelsize=15)
legend = []
for i in range(len(z_)):
    plt.scatter(mass, diff[:][i])
    legend.append(str(round(z_nodes[i], 3)) + "< z <" + str(round(z_nodes[i + 1], 3)))
plt.legend(legend, loc=4, fontsize=10)
plt.grid(True, which="both")
plt.ylabel("|bias| CCL exact method to Numcosmo $(\%)$", fontsize=20)
plt.xlabel(r"$log_{10}(M)$", fontsize=20)
plt.ylim(1e-9, 1e-1)
plt.show()


# diff[i][j] is the percentual difference between Numcosmo and CCL cluster abundance in i-th z_bin and j-th mass_bin

# Un-binned approach

Compute $$\frac{d^2V(z_i)}{dz d\Omega}\frac{dn(M_i,z_i)}{d\log_{10}M_i}$$ for each masses and redshifts in a dark matter halo catalog

In [ ]:
# Generates cluster count catalog data
ncdata = Nc.DataClusterNCount.new(ca, "NcClusterRedshiftNodist", "NcClusterMassNodist")
rng = Ncm.RNG.pool_get("example_ca_sampling")
ncdata.init_from_sampling(mset, 5264.05 * (np.pi / 180) ** 2, rng)
ncdata.catalog_save("ca_nodist_unbinned_numcosmo.fits", True)

ncdata_fits = fits.open("ca_nodist_unbinned_numcosmo.fits")
# ncdata_fits.info()

ncdata_data = ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table.columns
ncdata_Table.sort("Z_TRUE")

display(ncdata_Table)

In [ ]:
# Using Numcosmo
t0 = time.time()
d2n = []
for i in ncdata_Table:
    d2n.append(ca.d2n(cosmo, cluster_z, cluster_m, i[1], i[0]))
d2n = np.array(d2n)

t1 = time.time()
print("time (seconds) = " + str(t1 - t0))
d2n

In [ ]:
# Using CCL
t0 = time.time()
log10M = np.array(ncdata_Table[1][:] / np.log(10))
multiplicity_exact = clc.multiplicity_function_individual_MZ(
    z=np.array(ncdata_Table[0][:]), logm=log10M, method="exact"
)
multiplicity_exact *= clc.sky_area / math.log(10)

t1 = time.time()
print("time (seconds) = " + str(t1 - t0))
print(multiplicity_exact)

# 22.31 segundos
# 19.23 segundos high precision

In [ ]:
diff = abs(100 * (multiplicity_exact - d2n) / d2n)

plt.figure(figsize=(14, 7))
plt.yscale("log")
plt.tick_params(axis="both", which="major", labelsize=15)
plt.scatter(ncdata_Table[0][:], diff, s=20, c=log10M)
plt.colorbar()
plt.grid(True, which="both")
plt.xlabel(r"$redshift$", fontsize=20)
plt.ylabel("|bias| CCL exact method to Numcosmo $(\%)$", fontsize=20)
plt.ylim(10e-10, 1)